In [1]:
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tslearn.utils import to_time_series_dataset
import biosppy.signals.ecg as ecg
import matplotlib.pyplot as plt
import neurokit2 as nk
import heartpy as hp
from statistics import mean
import functions
from multiprocessing import Pool
from datetime import time
from pandarallel import pandarallel
import functions
import scipy
import math
import pywt
import operator
from functions import *
import itertools
from sklearn.ensemble import ExtraTreesClassifier


In [2]:
folder = ""
raw = folder + "raw_data/"
x_train = pd.read_csv('x_train_preprocess.csv')
x_test = pd.read_csv('x_test_preprocess.csv')
y_train = pd.read_csv(raw + 'y_train.csv', index_col=['id'])
x_train = x_train.fillna(0)
x_test = x_test.fillna(0)

In [3]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5117 entries, 0 to 5116
Columns: 555 entries, Unnamed: 0 to nk_feat_13
dtypes: float64(554), int64(1)
memory usage: 21.7 MB


In [4]:
selector = SelectKBest(score_func=f_classif, k=150)
selector.fit(np.array(x_train), np.array(y_train).squeeze())
cols = selector.get_support(indices=True)

In [5]:
#scale data
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [9]:
from catboost import CatBoostClassifier
cat = CatBoostClassifier(objective= 'MultiClass',iterations=300, random_state=0,eval_metric='TotalF1:average=Micro')

In [10]:
#cross validation score
scores = cross_val_score(cat, x_train, np.ravel(y_train), cv=10, scoring='f1_micro',n_jobs=-1)
print(scores.mean())
print(scores.std())
#0.6884819135273974 StandardScaler
#0.7000122309197652 / 0.012079469299786984 MinMaxScaler


Learning rate set to 0.236581
Learning rate set to 0.236581
Learning rate set to 0.236581
Learning rate set to 0.236581
0:	learn: 0.6165038	total: 779ms	remaining: 3m 52s
1:	learn: 0.6214984	total: 1.28s	remaining: 3m 10s
2:	learn: 0.6282302	total: 1.86s	remaining: 3m 4s
3:	learn: 0.6351792	total: 2.19s	remaining: 2m 41s
Learning rate set to 0.236584
4:	learn: 0.6401737	total: 2.45s	remaining: 2m 24s
0:	learn: 0.6193268	total: 2.53s	remaining: 12m 35s
Learning rate set to 0.236581
5:	learn: 0.6462541	total: 3.06s	remaining: 2m 29s
1:	learn: 0.6269273	total: 3.08s	remaining: 7m 39s
6:	learn: 0.6466884	total: 3.69s	remaining: 2m 34s
2:	learn: 0.6282302	total: 3.67s	remaining: 6m 3s
3:	learn: 0.6397394	total: 4.56s	remaining: 5m 37s
7:	learn: 0.6527687	total: 4.57s	remaining: 2m 46s
Learning rate set to 0.236581
Learning rate set to 0.236581
Learning rate set to 0.236584
8:	learn: 0.6608035	total: 5.19s	remaining: 2m 47s
4:	learn: 0.6410423	total: 5.2s	remaining: 5m 6s
5:	learn: 0.6445168

In [11]:
import pickle
filename = 'models/catboost_model.sav'
pickle.dump(cat, open(filename, 'wb'))
